# Different Impacts from Different Formulations of Tropical Cyclones

This notebook goes through the calculation of the impact of tropical cyclones on New Zealand, changing the formulation of the hazard, to show how each one changes the impact calculation.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from climada.entity import LitPop

from climada.entity.entity_def import Entity
from climada.entity import ImpactFuncSet, ImpactFunc
from climada.engine import Impact
from climada.engine import ImpactCalc

from climada.hazard import TCTracks, TropCyclone, Centroids
from climada.entity import ImpfTropCyclone

/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


### Exposure and centroid

In [3]:
# NZ with resolution 10km
exp_nz = LitPop.from_countries(countries=['NZL'],fin_mode='norm',res_arcsec=300)

# Define the centroids from the exposures position
lat = exp_nz.gdf['latitude'].values
lon = exp_nz.gdf['longitude'].values
centrs = Centroids.from_lat_lon(lat, lon)
centrs.check()

2024-08-03 13:23:09,133 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,159 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,175 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,191 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,223 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,238 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:23:09,257 - climada.entity.exposures.litpop.gpw_populati

### Impact function

In [4]:
# impact function TC
impf_tc = ImpfTropCyclone.from_emanuel_usa()

# add the impact function to an Impact function set
impf_set = ImpactFuncSet([impf_tc])

# Get the hazard type and hazard id
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]

In [5]:
# Exposures: rename column and assign id
exp_nz.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_nz.gdf['impf_' + haz_type] = haz_id
exp_nz.check()

### Cyclones with raw track data

In [6]:
# Load historical tropical cyclone tracks from ibtracs over the North Atlantic basin between 2010-2012
ibtracks_nz = TCTracks.from_ibtracs_netcdf(provider='wellington', basin='SP', year_range=(2010, 2023), correct_pres=True)
print('num tracks hist:', ibtracks_nz.size)

# Using the tracks, compute the windspeed at the location of the centroids
haz_tc = TropCyclone.from_tracks(ibtracks_nz, centroids=centrs)
haz_tc.check()

2024-08-03 13:23:10,276 - climada.hazard.tc_tracks - WARNING - `correct_pres` is deprecated. Use `estimate_missing` instead.
2024-08-03 13:23:10,561 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ming/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2024-08-03 13:23:11,547 - climada.hazard.tc_tracks - WARNING - 94 storm events are discarded because no valid wind/pressure values have been found: 2010020S14147, 2010022S12160, 2010027S14184, 2010038S08194, 2010051S06195, ...


/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


num tracks hist: 47


In [7]:
# Compute impact
imp_tc = ImpactCalc(exp_nz, impf_set, haz_tc).impact()  # Do not save the results geographically resolved (only aggregate values)

# aai_agg is the Aggregated average annual impact
print(f"Aggregated over New Zealand, average annual impact: {imp_tc.aai_agg}")

Aggregated over New Zealand, average annual impact: 4.5621755663973166e-07


### Cyclone with time step 0.5h

In [12]:
# Load historical tropical cyclone tracks from ibtracs over the North Atlantic basin between 2010-2012
ibtracks_nz = TCTracks.from_ibtracs_netcdf(provider='wellington', basin='SP', year_range=(2010, 2023), correct_pres=True)
ibtracks_nz.equal_timestep(time_step_h=0.5)
print('num tracks hist:', ibtracks_nz.size)

# Using the tracks, compute the windspeed at the location of the centroids
haz_tc = TropCyclone.from_tracks(ibtracks_nz, centroids=centrs)
haz_tc.check()

2024-08-03 13:23:27,605 - climada.hazard.tc_tracks - WARNING - `correct_pres` is deprecated. Use `estimate_missing` instead.
2024-08-03 13:23:27,779 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ming/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2024-08-03 13:23:28,663 - climada.hazard.tc_tracks - WARNING - 94 storm events are discarded because no valid wind/pressure values have been found: 2010020S14147, 2010022S12160, 2010027S14184, 2010038S08194, 2010051S06195, ...


/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


num tracks hist: 47


In [13]:
# Compute impact
imp_tc = ImpactCalc(exp_nz, impf_set, haz_tc).impact()  # Do not save the results geographically resolved (only aggregate values)

# aai_agg is the Aggregated average annual impact
print(f"Aggregated over New Zealand, average annual impact: {imp_tc.aai_agg}")

Aggregated over New Zealand, average annual impact: 5.292302626546226e-07


### Cyclone with time step 0.1h

In [14]:
# Load historical tropical cyclone tracks from ibtracs over the North Atlantic basin between 2010-2012
ibtracks_nz = TCTracks.from_ibtracs_netcdf(provider='wellington', basin='SP', year_range=(2010, 2023), correct_pres=True)
ibtracks_nz.equal_timestep(time_step_h=0.1) # recommended by Chahan
print('num tracks hist:', ibtracks_nz.size)

# Using the tracks, compute the windspeed at the location of the centroids
haz_tc = TropCyclone.from_tracks(ibtracks_nz, centroids=centrs)
haz_tc.check()

2024-08-03 13:23:31,534 - climada.hazard.tc_tracks - WARNING - `correct_pres` is deprecated. Use `estimate_missing` instead.
2024-08-03 13:23:31,691 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ming/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2024-08-03 13:23:32,504 - climada.hazard.tc_tracks - WARNING - 94 storm events are discarded because no valid wind/pressure values have been found: 2010020S14147, 2010022S12160, 2010027S14184, 2010038S08194, 2010051S06195, ...


/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


num tracks hist: 47


In [15]:
# Compute impact
imp_tc = ImpactCalc(exp_nz, impf_set, haz_tc).impact()  # Do not save the results geographically resolved (only aggregate values)

# aai_agg is the Aggregated average annual impact
print(f"Aggregated over New Zealand, average annual impact: {imp_tc.aai_agg}")

Aggregated over New Zealand, average annual impact: 6.4610466008611e-07
